In [1]:
import pyhanabi

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

#import checkpointer
#import iteration_statistics
#import dqn_agent
#import gin.tf
import rl_env
import numpy as np
#import rainbow_agent
import tensorflow as tf
import random

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags
import statistics
import pandas as pd

import rl_env
#import run_paired_experiment

In [4]:
def print_encoded_observations(encoder, state, num_players):
    print("--- EncodedObservations ---")
    print("Observation encoding shape: {}".format(encoder.shape()))
    print("Current actual player: {}".format(state.cur_player()))
    for i in range(num_players):
      print("Encoded observation for player {}: {}".format(
          i, encoder.encode(state.observation(i))))
    print("--- EndEncodedObservations ---")

In [5]:
def get_encoded_observations(encoder, state, player_num):
    codes = encoder.encode(state.observation(player_num))
    return codes

In [6]:
def print_observation(observation):
    """Print some basic information about an agent observation."""
    print("--- Observation ---")
    print(observation)

    print("### Information about the observation retrieved separately ###")
    print("### Current player, relative to self: {}".format(
        observation.cur_player_offset()))
    print("### Observed hands: {}".format(observation.observed_hands()))
    print("### Card knowledge: {}".format(observation.card_knowledge()))
    print("### Discard pile: {}".format(observation.discard_pile()))
    print("### Fireworks: {}".format(observation.fireworks()))
    print("### Deck size: {}".format(observation.deck_size()))
    move_string = "### Last moves:"
    for move_tuple in observation.last_moves():
      move_string += " {}".format(move_tuple)
    print(move_string)
    print("### Information tokens: {}".format(observation.information_tokens()))
    print("### Life tokens: {}".format(observation.life_tokens()))
    print("### Legal moves: {}".format(observation.legal_moves()))
    print("--- EndObservation ---")

In [7]:
def print_state(state):
    """Print some basic information about the state."""
    print("")
    print("Current player: {}".format(state.cur_player()))
    print(state)

    # Example of more queries to provide more about this state. For
    # example, bots could use these methods to to get information
    # about the state in order to act accordingly.
    print("### Information about the state retrieved separately ###")
    print("### Information tokens: {}".format(state.information_tokens()))
    print("### Life tokens: {}".format(state.life_tokens()))
    print("### Fireworks: {}".format(state.fireworks()))
    print("### Deck size: {}".format(state.deck_size()))
    print("### Discard pile: {}".format(str(state.discard_pile())))
    print("### Player hands: {}".format(str(state.player_hands())))
    print("")

In [8]:
import random
from rl_env import Agent


class RandomAgent():
    def act(self, observation):
        """Act based on an observation."""
        return random.choice(observation.legal_moves())

In [9]:
#agent = RandomAgent()
agent = RandomAgent()

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [12]:
import copy


def make_torch_seed():
    while True:
        seed = torch.random.seed()
        if seed < 2**63:
            return seed

class Net(nn.Module):
    
    #TODO: Try other network architectures (smaller layers)

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        #self.conv1 = nn.Conv2d(2, 2, 13)
        #self.conv2 = nn.Conv2d(2, 2, 1)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(658, 512)  # 6*6 from image dimension
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 20)

    
    #TODO: Try other activation func
    def forward(self, x):
        # Max pooling over a (2, 2) window
        #x = F.max_pool2d(F.relu(self.conv1(x)), (1, 1))
        # If the size is a square you can only specify a single number
        #x = F.max_pool2d(F.relu(self.conv2(x)), 1)
        #x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    """
    def foward2(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    """
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


initial_seed = torch.random.seed()
net = Net()
# print(net)
# print(net.parameters())
# flattened = []
# total = 0
# for p in net.parameters():
#     print(p)
# #     print(p)
# #     print(p.shape)
#     f = torch.flatten(p)
#     print(f.shape)
#     length = len(f)
#     print(length)
#     total += length
#     for i in range(length):
#         flattened.append(f[i].item())
# #     print(f)
# #     flattened.append(list(f))
# # print(net.parameters().shape)
# # print(total)
# # print(flattened[0])
# # noise = torch.empty(total).normal_(mean=0,std=0.1)
# print(noise[0])

modification_seed = torch.random.seed()
flattened = []
with torch.no_grad():
    for p in net.parameters():
        print("Starting weights:")
        print(p)
        s = p.shape
        print("Noise vector:")
        noise = torch.empty(s).normal_(mean=0,std=0.01)
        print(noise)
        p += noise
        print("New weights:")
        print(p)
#     print(f)
#     flattened.append(list(f))
# print(net.parameters().shape)
# print(total)
# print(flattened[0])
# noise = torch.empty(t

print()
print("-=-=--=-=--=-=-=-=")
print()


net2 = copy.deepcopy(net)
# net2.load_state_dict(net.state_dict())

#Copy

    
    
torch.random.manual_seed(modification_seed)
with torch.no_grad():
    for p in net.parameters():
        print("Starting weights:")
        print(p)
        s = p.shape
        print("Noise vector:")
        noise = torch.empty(s).normal_(mean=0,std=0.01)
        print(noise)
        p += noise
        print("New weights:")
        print(p)
        
    
print("")
print("-=-=-=-=-=-=-=-=-=-=-=-=")
print("aaa")

with torch.no_grad():
    for p2 in net2.parameters():
        print(p2)

Starting weights:
Parameter containing:
tensor([[ 0.0386,  0.0086,  0.0307,  ...,  0.0011,  0.0002, -0.0385],
        [-0.0191,  0.0120,  0.0363,  ...,  0.0310,  0.0307, -0.0032],
        [ 0.0215,  0.0275,  0.0066,  ...,  0.0144, -0.0361,  0.0275],
        ...,
        [ 0.0002,  0.0370,  0.0010,  ..., -0.0008, -0.0150, -0.0211],
        [-0.0052,  0.0134, -0.0032,  ...,  0.0237,  0.0091,  0.0235],
        [ 0.0383,  0.0298, -0.0111,  ..., -0.0097, -0.0135, -0.0115]],
       requires_grad=True)
Noise vector:
tensor([[ 0.0064, -0.0125, -0.0165,  ...,  0.0038, -0.0107,  0.0017],
        [ 0.0049,  0.0080, -0.0077,  ...,  0.0050, -0.0012, -0.0186],
        [ 0.0051, -0.0123, -0.0057,  ...,  0.0043, -0.0137, -0.0058],
        ...,
        [-0.0121,  0.0079,  0.0080,  ...,  0.0207,  0.0291,  0.0158],
        [-0.0157, -0.0067, -0.0096,  ...,  0.0102,  0.0162,  0.0013],
        [ 0.0045, -0.0015,  0.0015,  ..., -0.0053,  0.0201, -0.0022]])
New weights:
Parameter containing:
tensor([[ 0.0450

In [13]:
def list_to_tensor(mylist):
    encoded_observations = np.array(mylist).reshape(1,658)
    tensor_observations = torch.FloatTensor(encoded_observations)
    return tensor_observations

In [14]:
#list_encoded_observations = get_encoded_observations(obs_encoder, state, 0)
#encoded_observations = np.array(list_encoded_observations).reshape(1,658)
#tensor_observations = torch.FloatTensor(encoded_observations)
#net(tensor_observations)

In [15]:
#input = tensor_observations
#out = net(input)
#print(out)
#net.zero_grad()
#out.backward(torch.randn(1, 6))

In [16]:
#import torch.optim as optim

#criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [17]:
class PytorchAgent:

    def __init__(self,game,net=None,mutation_sd=0.01):
        if net is None:
            self.net = Net()
        else:
            self.net = copy.deepcopy(net)
        self.game = game #TODO is there a way to factoor this out?
        self.obs_encoder = pyhanabi.ObservationEncoder(game, enc_type=pyhanabi.ObservationEncoderType.CANONICAL)
        self.fitness=0.0

    def act(self, state):
        with torch.no_grad():
            encoded_observations = get_encoded_observations(self.obs_encoder, state, state.cur_player())
            input_tensor = list_to_tensor(encoded_observations)
            output_tensor = self.net(input_tensor)
            weight = output_tensor.tolist()[0]#weight of 20 possible moves
            rank = sorted(range(len(weight)), key=lambda k: weight[k]) # rank of 20 possible moves in weight
            index = sorted(range(len(rank)), key=lambda k: rank[k]) # index of sorted rank of 20 possible moves
            observation = state.observation(state.cur_player())# need to make sure whether observation = state works
            for i in index:
#                 print("checking move:" + str(self.game.get_move(i)))
                if str(self.game.get_move(i)) in str(observation.legal_moves()):# Is there a way of not using str?
#                     print("valid")
                    return self.game.get_move(i)
#                 else:
#                     print("invalid, checking next possible move in priority list")
        #here you probably need to check if it is a valid action
        # There's a code in the HLE that gets the legal actions from a state (a "mask")
        # you can do mask*model.forward()
        # This will multiply all ILLEGAL actions by zero
    
    # If seed is none, create a new seed and append to self.seeds. Otherwise, just use the given seed, but not append
    #TODO: Try other values for mutation rate
    def mutate(self,sd=0.01,seed=None): #TODO: Posssibly try to ranomly change only a few of the parametes usign a mutation rate (low priorty)?
        if seed is None:
            seed = self.make_torch_seed()
        with torch.no_grad():
            torch.random.manual_seed(seed)
            for p in net.parameters():
                shape = p.shape
                noise = torch.empty(shape).normal_(mean=0.0,std=sd)
                p += noise
    
    def make_torch_seed(self):
        while True:
            seed = torch.random.seed()
            if seed < 2**63:
                return seed
        
"""
    def make_action(array):
        index = np.rgmax(array)
        return index


    def get_weights():
        #Pytorch code to get an array with all the weights of the network
        weights  = [] #np.zeros()

    def set_wegiths(weights):
        #Pytorch code to set the weigths
"""

'\n    def make_action(array):\n        index = np.rgmax(array)\n        return index\n\n\n    def get_weights():\n        #Pytorch code to get an array with all the weights of the network\n        weights  = [] #np.zeros()\n\n    def set_wegiths(weights):\n        #Pytorch code to set the weigths\n'

In [18]:
game_parameters = {"players": 2, "random_start_player": True}
game = pyhanabi.HanabiGame(game_parameters)
print(game.parameter_string(), end="")
obs_encoder = pyhanabi.ObservationEncoder(game, enc_type=pyhanabi.ObservationEncoderType.CANONICAL)
state = game.new_initial_state()

pyagent = PytorchAgent(game)

population_size = 5
num_games = 5
num_generations = 500
# one run of game
score=0

# population = [PytorchAgent(game) for p in range(population_size)]

    
while not state.is_terminal():
    if state.score() >=score:
        score = state.score()
    if state.cur_player() == pyhanabi.CHANCE_PLAYER_ID:
        state.deal_random_card()
        continue

    #print_state(state)

    #observation = state.observation(state.cur_player())
    #print_observation(observation)
    #print(observation.legal_moves())
    #print_encoded_observations(obs_encoder, state, game.num_players())
    #encoded_observations = get_encoded_observations(obs_encoder, state, state.cur_player())
    #mytensor = list_to_tensor(encoded_observations)
    #legal_moves = state.legal_moves()
    # print("")
    # print("Number of legal moves: {}".format(len(legal_moves)))
    current_life_token = state.life_tokens()
    current_fireworks = state.fireworks()
    move  = pyagent.act(state)
    # move = np.random.choice(legal_moves)
    # print("Chose random legal move: {}".format(move))

    state.apply_move(move)
print("game finished")
print(state.fireworks())
print(score)

random_start_player=true
seed=1859835478
max_life_tokens=3
hand_size=5
observation_type=1
max_information_tokens=8
ranks=5
colors=5
players=2
game finished
[0, 0, 0, 0, 0]
0


In [19]:
import sys

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [ ]:
from datetime import datetime
import time
import os
def play_game(game_parameters,agent):
    game = pyhanabi.HanabiGame(game_parameters)
    obs_encoder = pyhanabi.ObservationEncoder(game, enc_type=pyhanabi.ObservationEncoderType.CANONICAL)
    state = game.new_initial_state()
    score=0
    
    while not state.is_terminal():
        if state.score() >=score:
            score = state.score()
        if state.cur_player() == pyhanabi.CHANCE_PLAYER_ID:
            state.deal_random_card()
            continue


        current_life_token = state.life_tokens()
        current_fireworks = state.fireworks()
        move  = agent.act(state)

        state.apply_move(move)
    return score

game_parameters = {"players": 2, "random_start_player": True}
game = pyhanabi.HanabiGame(game_parameters)
print(game.parameter_string(), end="")
obs_encoder = pyhanabi.ObservationEncoder(game, enc_type=pyhanabi.ObservationEncoderType.CANONICAL)

# pyagent = PytorchAgent(game)

#TODO: Try different values for these hyperparameters
population_size = 100
num_games = 20
num_generations = 1000
# one run of game

#TODO: Try to load the initial parameters of a few of the individuals from our Rainbow AIIIDE experiments.
population = [PytorchAgent(game) for p in range(population_size)]
print(population[0])


for gen in range(num_generations):
    # Evaluate each agent
    for agent in population:
        total_score = 0.0
        for g in range(num_games):
            score = play_game(game_parameters,agent)
            total_score+=score
#             print("Gen {} game {} had score {}".format((gen,g,score)))
        agent.fitness = float(total_score)/float(num_games)
    
    # Sort by fitness
    population.sort(key = lambda x: x.fitness, reverse=True)
    
    new_population=[]
    
    # Create new population by discarding the bottom half, duplicating the top half, then mutating one of the copies
    for i, a in enumerate(population): #TODO: Try other strategies such as make parent selection more likely for high fitness parents, elitism for the top k agents, etc
        if i >= population_size/2:
            parent_index = int(i - population_size/2) #Here it could be something like random parent (weighted by fitness)
            parent = population[parent_index]
            child = PytorchAgent(game,parent.net)
            child.mutate()
            new_population.append(child)
        else:
            agent.mutate()
            new_population.append(agent)
    population = new_population
    current_time = datetime.now().strftime("%H:%M:%S")
#     print ("Population Size is ".format(get_size([x.seeds for x in population])))
#     print(get_size([x.seeds for x in population]))
#     print(get_size([x.net for x in population]))
#     for p in population:
#         n = p.net
#         print(get_size(n.parameters()))
#         for param in n.parameters():
#             print(" " + str(get_size(param)))
#             print(" " + str(len(param)))
#             print(" " +str(param.shape))
#             print(param)
    print('Time {}. Best of generation {} has fitness {}'.format(current_time,gen, population[0].fitness))
    cmd = 'ps -o pid,user,%mem,command -p ' + str(os.getpid())
    print(os.system(cmd))
    

random_start_player=true
seed=1336896321
max_life_tokens=3
hand_size=5
observation_type=1
max_information_tokens=8
ranks=5
colors=5
players=2
Time 14:47:14. Best of generation 0 has fitness 1.05
0


In [19]:
environment = rl_env.make('Hanabi-Full', num_players=2)
state = environment.reset()
while not state.is_terminal():
    if state.cur_player() == pyhanabi.CHANCE_PLAYER_ID:  
        state.deal_random_card()
        continue

AttributeError: 'dict' object has no attribute 'is_terminal'

In [ ]:
state

In [ ]:
encoded_observations

In [ ]:
net.parameters

In [ ]:
len(get_encoded_observations(obs_encoder, state, 0))

In [ ]:
tensor_observations = torch.FloatTensor(get_encoded_observations(obs_encoder, state, 0))
net(tensor_observations)

In [ ]:
list(net(tensor_observations))

In [ ]:
move_value = net(tensor_observations).tolist()[0]

In [ ]:
move_value.index(max(move_value))

In [ ]:
def get_encoded_observations(encoder, state, player_num):
    codes = encoder.encode(state.observation(player_num))
    return codes

In [ ]:
state.cur_player()

In [ ]:
encoded_observations = get_encoded_observations(obs_encoder, state, state.cur_player())
mytensor = list_to_tensor(encoded_observations)
legal_moves = state.legal_moves()

In [ ]:
state = game.new_initial_state()

In [ ]:
state.legal_moves()

In [ ]:
state.life_tokens()

In [ ]:
state.fireworks()

In [ ]:
type(state)

In [ ]:
type(observation)

In [ ]:
observation.legal_moves()

In [ ]:
environment = rl_env.make('Hanabi-Full', num_players=2)
state = environment.reset()


In [ ]:
type(state)

In [ ]:
state.keys()

In [ ]:
observations = environment.reset()
type(observations)

In [ ]:
game = pyhanabi.HanabiGame()
obs_encoder = pyhanabi.ObservationEncoder(game, enc_type=pyhanabi.ObservationEncoderType.CANONICAL)
print(game.parameter_string(), end="")
obs_encoder = pyhanabi.ObservationEncoder(game, enc_type=pyhanabi.ObservationEncoderType.CANONICAL)

In [ ]:
type(state)

In [ ]:
dir(state)

In [ ]:
state.legal_moves()

In [3]:
import numpy as np

In [4]:
state = np.array([[[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]])

In [9]:
len(state[0])

658

In [10]:
test = [76, 72, 67, 69, 67, 77, 69, 69, 70, 67, 68, 70, 68, 68, 70, 71, 68, 70, 71, 70, 68, 69, 69, 66, 71, 67, 68, 66, 69, 70, 70, 72, 69, 74, 68, 69, 71, 72, 71, 72, 68, 68, 69, 71, 75, 71, 73, 66, 67, 73, 68, 34, 71, 69, 76, 69, 68, 69, 65, 67, 67, 68, 72, 68, 69, 71, 68, 69, 75, 73, 70, 67, 68]

In [11]:
len(test)

73

In [13]:
test = [15, 7, 11, 9, 12, 5, 12, 27, 6, 24, 10, 12, 17, 12, 6, 13, 15, 10, 10, 8, 13, 18, 22, 35, 24, 5, 13, 17, 9, 6, 11, 12, 18, 10, 11, 18, 6, 30, 46, 17, 15, 6, 20, 5, 9, 7, 8, 18, 8, 19, 9, 5, 9, 5, 11, 8, 6, 17, 10, 16, 4, 3, 7, 15, 6, 16, 15, 13, 11, 10, 16, 10, 3, 16, 11, 4, 8, 4, 6, 12, 9, 12, 7, 4, 9, 10, 5, 13, 4, 4, 5, 16, 6, 18, 10, 4, 8, 6, 6, 12, 4, 12, 3, 13, 6, 11, 16, 7, 3, 3, 4, 4, 6, 8, 5, 9, 3, 14, 26, 4, 4, 6, 4, 8, 18, 5, 3, 3, 3, 18, 8, 3, 12, 5, 7, 6, 6, 11, 7, 7, 14, 23, 5, 12, 15, 13, 31, 11, 19, 3, 3, 20, 14, 8, 5, 63, 41, 47, 7, 15, 57, 13, 78, 51, 14, 43, 27, 36, 29, 29, 21, 12, 36, 27, 63, 27, 18, 66, 11, 80, 33, 10, 27, 19, 27, 6, 25, 34, 58, 59, 22, 32, 87, 84, 68, 61, 85, 33, 29, 84, 58, 83, 60, 79, 81, 11, 86, 18, 23, 33, 60, 55, 68, 47, 14, 87, 18, 15, 79, 32, 79, 45, 13, 72, 22, 25, 9, 79, 34, 44, 62, 29, 28, 82]

In [14]:
len(test)

234